In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
minor = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)
minor_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240318상장비주력test_stand.csv', index_col=0)

In [ ]:
# major.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

##### train_df, test_df 설정

In [25]:
train_df = minor[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [26]:
test_df = minor_test[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [27]:
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [29]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
y_train.value_counts()

0.0    3431
1.0      54
Name: 부실, dtype: int64

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성
under = RandomUnderSampler(random_state=4, sampling_strategy=0.3)

# RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
x_resampled, y_resampled = under.fit_resample(x_train, y_train)

y_resampled.value_counts()

0.0    180
1.0     54
Name: 부실, dtype: int64

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# x_resampled2, y_resampled2 = RandomOverSampler(random_state=4, sampling_strategy=1).fit_resample(x_resampled, y_resampled )
# print(x_train.shape)
# y_resampled2.value_counts()

(3485, 13)


0.0    540
1.0    540
Name: 부실, dtype: int64

#### 1. Tabnet

In [17]:
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

# TabNet 모델 생성
tabnet_clf = TabNetClassifier()

# DataFrame을 NumPy 배열로 변환
x_train_np = x_resampled.values
y_train_np = y_resampled.values
x_test_np = x_test.values

# 모델 훈련
tabnet_clf.fit(x_train_np, y_train_np)

# 테스트 데이터에 대한 예측
tabnet_pred = tabnet_clf.predict(x_test_np)
tabnet_pred_proba = tabnet_clf.predict_proba(x_test_np)

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

In [ ]:
# 성능 평가
print("TabNet")
get_clf_eval(y_test, tabnet_pred)

TabNet
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000


In [ ]:
# from pytorch_tabnet.tab_model import TabNetClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer, accuracy_score

# # TabNet 모델 정의
# tabnet_clf = TabNetClassifier()

# # 그리드 서치를 위한 하이퍼파라미터 그리드 설정
# param_grid = {
#     'n_d': [8, 16, 32],  # Decision 단위의 피처 개수
#     'n_a': [8, 16, 32],  # Attention 단위의 피처 개수
#     'n_steps': [3, 5, 10],  # 역전파 단계 수
#     'gamma': [1.0, 1.5, 2.0],  # 역전파 시 감마 값
#     'lambda_sparse': [0.0001, 0.001, 0.01],  # 희소 손실 정규화 파라미터
# }

# # 그리드 서치 수행
# grid_search = GridSearchCV(tabnet_clf, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train_np, y_train_np)

# # 최적의 모델과 하이퍼파라미터 가져오기
# best_tabnet_clf = grid_search.best_estimator_
# best_params = grid_search.best_params_

# # 테스트 데이터에 대한 예측 수행
# tabnet_pred = best_tabnet_clf.predict(x_test)
# tabnet_pred_proba = best_tabnet_clf.predict_proba(x_test)

In [ ]:
# # 분류 임계값 조정
# threshold = 0.5  # 임계값 설정
# tabnet_pred_adjusted = (tabnet_pred_proba[:, 1] >= threshold).astype(int)

# 임곗값 조정 후 성능 평가
# print("TabNet")
# get_clf_eval(y_test, tabnet_pred)

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(y_true, y_pred):
    # 정확도 (Accuracy)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

    # 정밀도 (Precision)
    precision = precision_score(y_true, y_pred)
    print("Precision:", precision)

    # 재현율 (Recall)
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # F1 스코어 (F1 Score)
    f1 = f1_score(y_true, y_pred)
    print("F1 Score:", f1)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_true, y_pred)
        print("AUC-ROC:", auc_roc)
    except ValueError:
        print("AUC-ROC cannot be calculated for multiclass classification.")

#### 2. LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Model Building
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(x_resampled.shape[1],1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Model Compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(x_resampled, y_resampled, epochs=100, batch_size=64)

# 모델 평가
y_pred = model.predict(x_test)
predictions_binary = (y_pred > 0.5).astype(int)

evaluate(y_test, predictions_binary)

Epoch 1/100
4/4 [==============================] - 2s 26ms/step - loss: 0.6829 - accuracy: 0.7607
Epoch 2/100
4/4 [==============================] - 0s 24ms/step - loss: 0.6657 - accuracy: 0.7692
Epoch 3/100
4/4 [==============================] - 0s 25ms/step - loss: 0.6474 - accuracy: 0.7692
Epoch 4/100
4/4 [==============================] - 0s 24ms/step - loss: 0.6248 - accuracy: 0.7692
Epoch 5/100
4/4 [==============================] - 0s 24ms/step - loss: 0.5885 - accuracy: 0.7735
Epoch 6/100
4/4 [==============================] - 0s 29ms/step - loss: 0.5320 - accuracy: 0.7906
Epoch 7/100
4/4 [==============================] - 0s 29ms/step - loss: 0.4501 - accuracy: 0.8077
Epoch 8/100
4/4 [==============================] - 0s 25ms/step - loss: 0.3549 - accuracy: 0.8376
Epoch 9/100
4/4 [==============================] - 0s 23ms/step - loss: 0.3244 - accuracy: 0.8547
Epoch 10/100
4/4 [==============================] - 0s 29ms/step - loss: 0.2883 - accuracy: 0.8803
Epoch 11/100
4/4 [=

In [ ]:
# 임계값에 따라 예측값 조정
threshold = 0.3 # 기본 임계값

predictions_binary = (y_pred > threshold).astype(int)
evaluate(y_test, predictions_binary)

Accuracy: 0.898972602739726
Precision: 0.04065040650406504
Recall: 1.0
F1 Score: 0.078125
AUC-ROC: 0.9492691315563199


#### 3. CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# 모델 생성
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(x_resampled.shape[1],1)))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_resampled, y_resampled, epochs=100, batch_size=64)

# 모델 평가
y_pred = model.predict(x_test)

predictions_binary = (y_pred > 0.5).astype(int)
evaluate(y_test, predictions_binary)

Epoch 1/100
4/4 [==============================] - 1s 8ms/step - loss: 0.6464 - accuracy: 0.7735
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: 0.5629 - accuracy: 0.8504
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: 0.4974 - accuracy: 0.8675
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 0.4522 - accuracy: 0.8761
Epoch 5/100
4/4 [==============================] - 0s 6ms/step - loss: 0.4111 - accuracy: 0.8675
Epoch 6/100
4/4 [==============================] - 0s 6ms/step - loss: 0.3763 - accuracy: 0.8675
Epoch 7/100
4/4 [==============================] - 0s 6ms/step - loss: 0.3332 - accuracy: 0.8761
Epoch 8/100
4/4 [==============================] - 0s 6ms/step - loss: 0.3100 - accuracy: 0.8889
Epoch 9/100
4/4 [==============================] - 0s 6ms/step - loss: 0.2790 - accuracy: 0.8846
Epoch 10/100
4/4 [==============================] - 0s 6ms/step - loss: 0.2630 - accuracy: 0.9017
Epoch 11/100
4/4 [===========

In [ ]:
# 임계값에 따라 예측값 조정
threshold = 0.3 # 기본 임계값

predictions_binary = (y_pred > threshold).astype(int)
evaluate(y_test, predictions_binary)

Accuracy: 0.9332191780821918
Precision: 0.060240963855421686
Recall: 1.0
F1 Score: 0.11363636363636363
AUC-ROC: 0.9664660361134996


#### 교차검증

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(x_train, y_train)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # undersampling 수행
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_fold, y_train_fold)

    X_train_resampled = X_train_resampled.values.reshape(X_train_resampled.shape[0], 1, X_train_resampled.shape[1])
    y_train_resampled = y_train_resampled.values

    # 모델 생성
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=(X_train_resampled.shape[1:])))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, verbose=0)  # verbose=0으로 설정하여 학습 과정 출력 생략

    # 검증 데이터에 대한 예측 및 평가
    X_val_fold = X_val_fold.values.reshape(X_val_fold.shape[0], 1, X_val_fold.shape[1])
    y_pred = model.predict(X_val_fold)

    # 이진 분류로 변환
    y_pred_binary = (y_pred > 0.3).astype(int)

    evaluate(y_val_fold,y_pred_binary)

Fold 1/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.9583931133428981
Precision: 0.1724137931034483
Recall: 0.5
F1 Score: 0.25641025641025644
AUC-ROC: 0.7325327510917031
Fold 2/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.945480631276901
Precision: 0.18604651162790697
Recall: 0.7272727272727273
F1 Score: 0.2962962962962963
AUC-ROC: 0.838126159554731
Fold 3/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.9598278335724534
Precision: 0.2571428571428571
Recall: 0.8181818181818182
F1 Score: 0.39130434782608686
AUC-ROC: 0.8901404717731249
Fold 4/5
22/22 [==============================] - 0s 4ms/step
Accuracy: 0.9497847919655668
Precision: 0.2
Recall: 0.7272727272727273
F1 Score: 0.3137254901960785
AUC-ROC: 0.8403127484760137
Fold 5/5
22/22 [==============================] - 0s 4ms/step
Accuracy: 0.945480631276901
Precision: 0.22448979591836735
Recall: 1.0
F1 Score: 0.36666666666666664
AUC-ROC: 0.9723032069970846


In [19]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

feature =x_train
label = y_train

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(feature, label)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = feature.iloc[train_idx], feature.iloc[val_idx]
    y_train_fold, y_val_fold = label.iloc[train_idx], label.iloc[val_idx]

    # undersampling 수행
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_fold, y_train_fold)

    X_train_resampled = X_train_resampled.values.reshape(X_train_resampled.shape[0], X_train_resampled.shape[1], 1)
    y_train_resampled = y_train_resampled.values

    # 모델 생성
    model = Sequential()
    model.add(Conv1D(128, 3, activation='relu', input_shape=(X_train_resampled.shape[1:])))
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 학습
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, verbose=0)  # verbose=0으로 설정하여 학습 과정 출력 생략

    # 검증 데이터에 대한 예측 및 평가
    X_val_fold = X_val_fold.values.reshape(X_val_fold.shape[0], X_val_fold.shape[1], 1)
    y_pred = model.predict(X_val_fold)
    predictions_binary = (y_pred > 0.3).astype(int)
    evaluate(y_val_fold, predictions_binary)

Fold 1/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.9497847919655668
Precision: 0.16216216216216217
Recall: 0.6
F1 Score: 0.25531914893617025
AUC-ROC: 0.7774381368267831
Fold 2/5
22/22 [==============================] - 0s 3ms/step
Accuracy: 0.9540889526542324
Precision: 0.21621621621621623
Recall: 0.7272727272727273
F1 Score: 0.33333333333333337
AUC-ROC: 0.8424993373972967
Fold 3/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.9512195121951219
Precision: 0.23255813953488372
Recall: 0.9090909090909091
F1 Score: 0.3703703703703703
AUC-ROC: 0.9304929764113438
Fold 4/5
22/22 [==============================] - 0s 3ms/step
Accuracy: 0.9512195121951219
Precision: 0.1891891891891892
Recall: 0.6363636363636364
F1 Score: 0.2916666666666667
AUC-ROC: 0.7963159289689901
Fold 5/5
22/22 [==============================] - 0s 2ms/step
Accuracy: 0.9354375896700143
Precision: 0.19642857142857142
Recall: 1.0
F1 Score: 0.3283582089552239
AUC-ROC: 0.967201166180758


In [20]:
!pip install pytorch_tabnet.tab_model

ERROR: Could not find a version that satisfies the requirement pytorch_tabnet.tab_model (from versions: none)
ERROR: No matching distribution found for pytorch_tabnet.tab_model


In [30]:
from pytorch_tabnet.tab_model import TabNetClassifier
from imblearn.under_sampling import RandomUnderSampler
import torch

X= x_train
y=y_train

rus = RandomUnderSampler(sampling_strategy=0.1)
X_resampled, y_resampled = rus.fit_resample(X, y)

X = X_resampled.values
y = y_resampled.values

# x_test, y_test = label_split(test_s)

X_test = x_test.values
y_test = y_test.values

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # 모델 생성
    clf = TabNetClassifier(
        n_d=8, n_a=8,
        optimizer_fn=torch.optim.Adam,
        scheduler_fn=torch.optim.lr_scheduler.CosineAnnealingLR,
        scheduler_params={'T_max': 100}
    )

    # 모델 학습
    clf.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric=['accuracy'],
        max_epochs=100,
        patience=20,  # 조기 종료를 위한 설정
        batch_size=1024,  # 배치 사이즈 설정
        virtual_batch_size=128,  # 가상 배치 사이즈 설정
        num_workers=0,  # 사용할 워커 수 설정
        drop_last=False
    )

    # 검증 데이터에 대한 예측
    y_pred = clf.predict(X_test)
    evaluate(y_test, y_pred)

Fold 1/5
epoch 0  | loss: 0.53096 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 1  | loss: 0.41422 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 2  | loss: 0.35532 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 3  | loss: 0.36587 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 4  | loss: 0.30267 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 5  | loss: 0.29398 | val_0_accuracy: 0.90756 |  0:00:00s
epoch 6  | loss: 0.293   | val_0_accuracy: 0.90756 |  0:00:01s
epoch 7  | loss: 0.26372 | val_0_accuracy: 0.90756 |  0:00:01s
epoch 8  | loss: 0.24227 | val_0_accuracy: 0.90756 |  0:00:01s
epoch 9  | loss: 0.23643 | val_0_accuracy: 0.90756 |  0:00:01s
epoch 10 | loss: 0.24521 | val_0_accuracy: 0.91597 |  0:00:01s
epoch 11 | loss: 0.20795 | val_0_accuracy: 0.90756 |  0:00:01s
epoch 12 | loss: 0.20676 | val_0_accuracy: 0.91597 |  0:00:01s
epoch 13 | loss: 0.20082 | val_0_accuracy: 0.91597 |  0:00:02s
epoch 14 | loss: 0.20401 | val_0_accuracy: 0.90756 |  0:00:02s
epoch 15 | loss: 0.21116 | val_0_accuracy: 0.8